In [1]:
import numpy as np
np.set_printoptions(precision=3)
sigma = lambda x: 1/(1+np.exp(-x))
d_sigma= lambda x: sigma(x)*(1-sigma(x))
eta=4.3
def F_prop(X, W):
    L=len(W)
    F_X = {'X0':X.T}
    for h in range(1, L+1):
        F_X[f'Y{h}']=np.dot(W[f'W{h}'], F_X[f'X{h-1}'])
        if h!=L:
            F_X[f'X{h}']=np.concatenate((sigma(F_X[f'Y{h}']), np.ones((1,X.shape[0]))), axis = 0)
        else:
            F_X[f'X{h}']=sigma(F_X[f'Y{h}'])
    return F_X

def F_star_W(X,y, W):
    L=len(W)
    F_X = F_prop(X, W)
    FX_star={'X*2':(F_X['X2']-y)}
    Y_star={}
    ##
    FX_star['Y*2']=FX_star['X*2']*(d_sigma(F_X['Y2']))    
    W['W2']= W['W2'] -eta*FX_star['Y*2'].dot(F_X['X1'].T) 
    ##
    FX_star['X*1']=np.delete(W['W2'], -1, axis = 1).T.dot(FX_star['Y*2'])
    FX_star['Y*1']=FX_star['X*1']*(d_sigma(F_X['Y1']))
    W['W1']= W['W1'] -eta*FX_star['Y*1'].dot(F_X['X0'].T)     
    return W
def F_star(X,y, W):
    L=len(W)
    F_X = F_prop(X, W)
    FX_star={'X*2':(F_X['X2']-y)}      
    FX_star['Y*2']=FX_star['X*2']*(d_sigma(F_X['Y2']))
    ##
    W['W2']= W['W2'] -eta*FX_star['Y*2'].dot(F_X['X1'].T) 
    #
    FX_star['X*1']=np.delete(W['W2'], -1, axis = 1).T.dot(FX_star['Y*2'])
    FX_star['Y*1']=FX_star['X*1']*(d_sigma(F_X['Y1']))        
    return FX_star

def F_star_prop(X,Y, W):
    L=len(W)
    F_X = F_prop(X, W)
    FX_star={f'X*{L}':(F_X[f'X{L}']-Y)}
    Y_star={}
    for h in reversed(range(1, L+1)):
        FX_star[f'Y*{h}']=FX_star[f'X*{h}']*(d_sigma(F_X[f'Y{h}']))
        FX_star[f'X*{h-1}']=np.delete(W[f'W{h}'], -1, axis = 1).T.dot(FX_star[f'Y*{h}'])
    return FX_star
def Grad_star(X,Y, W):
    L=len(W)
    F_X = F_prop(X, W)
    FX_star=F_star_prop(X,Y, W)
    Grad={}
    for h in range(1, L+1):
        Grad[f'W{h}']=W[f'W{h}']-eta*FX_star[f'Y*{h}'].dot(F_X[f'X{h-1}'].T)
    return Grad

In [2]:
# Learning the XOR dataset
X = np.array([[0, 0, 1], [0, 1, 1], [1, 0, 1], [1, 1, 1]])
y = np.array([0, 1, 1, 0])          
# Neural network with the architecture A[3,2,1]
W1=np.array([[1, 0, 0], [0, 1, 0]])
W2=np.array([[0, 0, 1]])
W={'W1':W1, 'W2':W2}
##
X2=F_prop(X, W)['X2']
X2_c=np.where(X2>=0.5,1,0)
k=0
while (X2_c -y).any() != 0 :
   # print("================================")
    print(W, X2, X2_c,k)
    print("================================================================================================")
    W=F_star_W(X,y, W)
    X2=F_prop(X, W)['X2']
    X2_c=np.where(X2>=0.5,1,0)
    k+=1

{'W1': array([[1, 0, 0],
       [0, 1, 0]]), 'W2': array([[0, 0, 1]])} [[0.731 0.731 0.731 0.731]] [[1 1 1 1]] 0
{'W1': array([[1.037, 0.031, 0.084],
       [0.031, 1.037, 0.084]]), 'W2': array([[-0.481, -0.481,  0.219]])} [[0.43  0.402 0.402 0.375]] [[0 0 0 0]] 1
{'W1': array([[1.027, 0.013, 0.066],
       [0.013, 1.027, 0.066]]), 'W2': array([[-0.211, -0.211,  0.625]])} [[0.6   0.588 0.588 0.577]] [[1 1 1 1]] 2
{'W1': array([[1.042, 0.016, 0.101],
       [0.016, 1.042, 0.101]]), 'W2': array([[-0.442, -0.442,  0.257]])} [[0.449 0.423 0.423 0.398]] [[0 0 0 0]] 3
{'W1': array([[ 1.033, -0.001,  0.086],
       [-0.001,  1.033,  0.086]]), 'W2': array([[-0.224, -0.224,  0.582]])} [[0.586 0.573 0.573 0.561]] [[1 1 1 1]] 4
{'W1': array([[ 1.045, -0.002,  0.114],
       [-0.002,  1.045,  0.114]]), 'W2': array([[-0.419, -0.419,  0.274]])} [[0.458 0.434 0.434 0.41 ]] [[0 0 0 0]] 5
{'W1': array([[ 1.038, -0.018,  0.101],
       [-0.018,  1.038,  0.101]]), 'W2': array([[-0.231, -0.231,  0.554]])}

In [3]:
# Learning the XOR dataset
X = np.array([[0, 0, 1], [0, 1, 1], [1, 0, 1], [1, 1, 1]])
y = np.array([0, 1, 1, 0])
# Neural network with the architecture A[3,2,1]
W1=np.array([[1, 0, 0], [0, 1, 0]])
W2=np.array([[0, 0, 1]])
W={'W1':W1, 'W2':W2}
##
X2=F_prop(X, W)['X2']
X2_c=np.where(X2>=0.5,1,0)
k=0
while (X2_c -y).any() != 0 :
   # print("================================")
    print(W, X2, X2_c,k)
    print("================================================================================================")
    W=Grad_star(X,y, W)
    X2=F_prop(X, W)['X2']
    X2_c=np.where(X2>=0.5,1,0)
    k+=1

{'W1': array([[1, 0, 0],
       [0, 1, 0]]), 'W2': array([[0, 0, 1]])} [[0.731 0.731 0.731 0.731]] [[1 1 1 1]] 0
{'W1': array([[1., 0., 0.],
       [0., 1., 0.]]), 'W2': array([[-0.481, -0.481,  0.219]])} [[0.435 0.408 0.408 0.381]] [[0 0 0 0]] 1
{'W1': array([[ 0.978, -0.037, -0.04 ],
       [-0.037,  0.978, -0.04 ]]), 'W2': array([[-0.236, -0.236,  0.603]])} [[0.592 0.579 0.579 0.566]] [[1 1 1 1]] 2
{'W1': array([[ 0.986, -0.034, -0.022],
       [-0.034,  0.986, -0.022]]), 'W2': array([[-0.434, -0.434,  0.272]])} [[0.461 0.437 0.437 0.413]] [[0 0 0 0]] 3
{'W1': array([[ 0.973, -0.061, -0.047],
       [-0.061,  0.973, -0.047]]), 'W2': array([[-0.266, -0.266,  0.54 ]])} [[0.57  0.556 0.556 0.541]] [[1 1 1 1]] 4
{'W1': array([[ 0.979, -0.06 , -0.032],
       [-0.06 ,  0.979, -0.032]]), 'W2': array([[-0.405, -0.405,  0.305]])} [[0.477 0.455 0.455 0.433]] [[0 0 0 0]] 5
{'W1': array([[ 0.97 , -0.081, -0.048],
       [-0.081,  0.97 , -0.048]]), 'W2': array([[-0.284, -0.284,  0.498]])} [[0.5